In [1]:
import re
import time
import pandas

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import Select

In [3]:
options = Options()
options.add_experimental_option("detach", True)
options.add_argument('start-maximized')

# 사람인척하기위해서 개발자도구 network에서 user-agent추가, 언어 추가
options.add_argument('Chrome/134.0.0.0')
options.add_argument('lang=ko_KR')

# gpu 안쓰기
options.add_argument('--disable-gpu') 

In [4]:
driver = webdriver.Chrome(
service=Service(ChromeDriverManager().install()),
options=options
)

driver.get(f'https://www.koreabaseball.com/Schedule/Schedule.aspx')

In [5]:
month_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#ddlMonth"))
        )
month_select = Select(month_btn)
month_select.select_by_value('06')

In [6]:
table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#tblScheduleList"))
        )
table_html = table.get_attribute('outerHTML')

In [7]:
table_html

'<table id="tblScheduleList" class="tbl" summary="경기일정 및 결과 표 (날짜, 시간, 경기, 게임센터, 하이라이트, TV, 라디오, 구장, 비고)">\n\t\t\t    <caption>경기일정 및 결과 표</caption>\n\t\t\t    <colgroup>\n\t\t\t\t    <col width="10%">\n\t\t\t\t    <col width="7%">\n\t\t\t\t    <col width="25%">\n\t\t\t\t    <col width="12%">\n\t\t\t\t    <col width="12%">\n\t\t\t\t    <col width="7%">\n\t\t\t\t    <col width="8%">\n\t\t\t\t    <col width="7%">\n\t\t\t\t    <col width="12%">\n\t\t\t    </colgroup>\n\t\t\t    <thead>\n\t\t\t\t    <tr>\n\t\t\t\t\t    <th>날짜</th>\n\t\t\t\t\t    <th>시간</th>\n\t\t\t\t\t    <th>경기</th>\n\t\t\t\t\t    <th>게임센터</th>\n\t\t\t\t\t    <th>하이라이트</th>\n\t\t\t\t\t    <th>TV</th>\n\t\t\t\t\t    <th>라디오</th>\n\t\t\t\t\t    <th>구장</th>\n\t\t\t\t\t    <th>비고</th>\n\t\t\t\t    </tr>\n\t\t\t    </thead>\n\t\t    <tbody><tr><td class="day" rowspan="5">06.01(일)</td><td class="time"><b>14:00</b></td><td class="play"><span>두산</span><em><span class="lose">0</span><span>vs</span><span class="win">1</span></em><s

In [8]:
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup

soup = BeautifulSoup(table_html, "html.parser")
df = pd.read_html(StringIO(str(soup)))[0]

df

,날짜,시간,경기,게임센터,하이라이트,TV,라디오,구장,비고
0,06.01(일),14:00,두산0vs1키움,리뷰,하이라이트,S-T,NaN,고척,-
1,06.01(일),17:00,삼성6vs4LG,리뷰,하이라이트,SPO-T,NaN,잠실,-
2,06.01(일),17:00,SSG4vs3롯데,리뷰,하이라이트,MS-T SS-T,NaN,사직,-
3,06.01(일),17:00,한화5vs16NC,리뷰,하이라이트,KN-T,NaN,창원,-
4,06.01(일),17:00,KIA5vs3KT,리뷰,하이라이트,SPO-2T,NaN,수원,-
...,...,...,...,...,...,...,...,...,...
120,06.29(일),14:00,삼성vs키움,프리뷰,NaN,SS-T,NaN,고척,-
121,06.29(일),17:00,KIAvsLG,프리뷰,NaN,MS-T,NaN,잠실,-
122,06.29(일),17:00,한화vsSSG,프리뷰,NaN,KN-T,NaN,문학,-
123,06.29(일),17:00,KTvs롯데,프리뷰,NaN,SPO-2T,NaN,사직,-


In [9]:
# 필요없는 열 삭제
df.drop(['게임센터', '하이라이트', 'TV', '라디오'], axis=1, inplace=True)

# 필요없는 행 삭제
drop_row = df[df['비고']!='-'].index
df.drop(drop_row, inplace=True)
df.reset_index(drop=True, inplace=True)

In [10]:
df.head()

,날짜,시간,경기,구장,비고
0,06.01(일),14:00,두산0vs1키움,고척,-
1,06.01(일),17:00,삼성6vs4LG,잠실,-
2,06.01(일),17:00,SSG4vs3롯데,사직,-
3,06.01(일),17:00,한화5vs16NC,창원,-
4,06.01(일),17:00,KIA5vs3KT,수원,-


In [11]:
# 날짜 컬럼 분해 함수
def split_date_columns(df):
    df.insert(0, '월', df['날짜'].str[:2])
    df.insert(1, '일',df['날짜'].str[3:5])
    df.insert(2, '요일',df['날짜'].str[6])
    df.drop('날짜', axis=1, inplace=True)
    return df

In [12]:
# 경기 컬럼 정규표현식 분해 함수
# 홈팀승리 1, 무승부 0, 홈팀패배 -1
def parse_game_info(game_str):
    pattern = r'([가-힣A-Z]+)(\d+)vs(\d+)([가-힣A-Z]+)'
    match = re.match(pattern, game_str)
    if match:
        home_team, home_score, away_score, away_team = match.groups()
        home_score = int(home_score)
        away_score = int(away_score)

        # 무승부 반영
        if home_score > away_score:
            home_result = 1
        elif home_score < away_score:
            home_result = -1
        else:
            home_result = 0

        return pd.Series([home_team, home_score, away_score, away_team, home_result])
    else:
        return pd.Series([None]*5)

In [13]:
split_date_columns(df)

,월,일,요일,시간,경기,구장,비고
0,06,01,일,14:00,두산0vs1키움,고척,-
1,06,01,일,17:00,삼성6vs4LG,잠실,-
2,06,01,일,17:00,SSG4vs3롯데,사직,-
3,06,01,일,17:00,한화5vs16NC,창원,-
4,06,01,일,17:00,KIA5vs3KT,수원,-
...,...,...,...,...,...,...,...
106,06,29,일,14:00,삼성vs키움,고척,-
107,06,29,일,17:00,KIAvsLG,잠실,-
108,06,29,일,17:00,한화vsSSG,문학,-
109,06,29,일,17:00,KTvs롯데,사직,-


In [14]:
df[['홈팀', '홈점수', '원정점수', '원정팀', '홈팀승리']] = df['경기'].apply(parse_game_info)
df = df.drop(['경기', '비고'], axis=1)
df

,월,일,요일,시간,구장,홈팀,홈점수,원정점수,원정팀,홈팀승리
0,06,01,일,14:00,고척,두산,0.0,1.0,키움,-1.0
1,06,01,일,17:00,잠실,삼성,6.0,4.0,LG,1.0
2,06,01,일,17:00,사직,SSG,4.0,3.0,롯데,1.0
3,06,01,일,17:00,창원,한화,5.0,16.0,NC,-1.0
4,06,01,일,17:00,수원,KIA,5.0,3.0,KT,1.0
...,...,...,...,...,...,...,...,...,...,...
106,06,29,일,14:00,고척,None,NaN,NaN,None,NaN
107,06,29,일,17:00,잠실,None,NaN,NaN,None,NaN
108,06,29,일,17:00,문학,None,NaN,NaN,None,NaN
109,06,29,일,17:00,사직,None,NaN,NaN,None,NaN


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   월       111 non-null    object 
 1   일       111 non-null    object 
 2   요일      111 non-null    object 
 3   시간      111 non-null    object 
 4   구장      111 non-null    object 
 5   홈팀      96 non-null     object 
 6   홈점수     96 non-null     float64
 7   원정점수    96 non-null     float64
 8   원정팀     96 non-null     object 
 9   홈팀승리    96 non-null     float64
dtypes: float64(3), object(7)
memory usage: 8.8+ KB


In [16]:
driver.quit()

## 코드 합치기

In [17]:
options = Options()
options.add_experimental_option("detach", True)
options.add_argument('start-maximized')

# 사람인척하기위해서 개발자도구 network에서 user-agent추가, 언어 추가
options.add_argument('Chrome/134.0.0.0')
options.add_argument('lang=ko_KR')

# gpu 안쓰기
options.add_argument('--disable-gpu') 

In [18]:
def fetch_2025_kbo_results(month):
    
    # selenium으로 KBO 공식 홈페이지 접속
    driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=options
            )
    driver.get(f'https://www.koreabaseball.com/Schedule/Schedule.aspx')
    
    # 경기 기록 table 추출
    month_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#ddlMonth"))
            )
    month_select = Select(month_btn)
    month_select.select_by_value(str(month).zfill(2))
    table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#tblScheduleList"))
            )
    table_html = table.get_attribute('outerHTML')
    driver.quit()
    
    # HTML을 데이터프레임으로!
    soup = BeautifulSoup(table_html, "html.parser")
    df = pd.read_html(StringIO(str(soup)))[0]
    
    split_date_columns(df)
    df[['홈팀', '홈점수', '원정점수', '원정팀', '홈팀승리']] = df['경기'].apply(parse_game_info)

    # 필요없는 행 삭제
    drop_row = df[df['하이라이트'].isna()].index
    df.drop(drop_row, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # 필요없는 열 삭제
    df.drop(['게임센터', '하이라이트', 'TV', '라디오', '경기', '비고'], axis=1, inplace=True)
    
    return df

In [19]:
pd.set_option('display.float_format', '{:.0f}'.format)

In [20]:
pd.set_option('display.max_rows',1000)

In [21]:
dfs = []

for i in range(3,7):
    df = fetch_2025_kbo_results(i)
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)
display(merged_df)

,월,일,요일,시간,구장,홈팀,홈점수,원정점수,원정팀,홈팀승리
0,03,22,토,14:00,잠실,롯데,2,12,LG,-1
1,03,22,토,14:00,문학,두산,5,6,SSG,-1
2,03,22,토,14:00,대구,키움,5,13,삼성,-1
3,03,22,토,14:00,수원,한화,4,3,KT,1
4,03,22,토,14:00,광주,NC,2,9,KIA,-1
5,03,23,일,14:00,잠실,롯데,2,10,LG,-1
6,03,23,일,14:00,문학,두산,2,5,SSG,-1
7,03,23,일,14:00,대구,키움,7,11,삼성,-1
8,03,23,일,14:00,수원,한화,4,5,KT,-1
9,03,23,일,14:00,광주,NC,5,4,KIA,1


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   월       96 non-null     object 
 1   일       96 non-null     object 
 2   요일      96 non-null     object 
 3   시간      96 non-null     object 
 4   구장      96 non-null     object 
 5   홈팀      96 non-null     object 
 6   홈점수     96 non-null     float64
 7   원정점수    96 non-null     float64
 8   원정팀     96 non-null     object 
 9   홈팀승리    96 non-null     float64
dtypes: float64(3), object(7)
memory usage: 7.6+ KB


In [25]:
import os
import pandas as pd

# 1. 상위 폴더 기준 경로 계산
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(parent_dir, 'data')

# 2. data 폴더 없으면 생성
os.makedirs(data_dir, exist_ok=True)

# 3. 파일 경로 설정
file_path = os.path.join(data_dir, '2025_KBO_승패기록.xlsx')

# 4. 저장
# merged_df.to_csv(file_path, encoding='utf-8-sig', index=False)
merged_df.to_excel(file_path, index=False)

print(f"파일이 저장되었습니다: {file_path}")

파일이 저장되었습니다: C:\KBO-WinPredictor\data\2025_KBO_승패기록.xlsx


# 기상청 API에서 날씨 정보

In [60]:
import os
from dotenv import load_dotenv, find_dotenv

# .env 파일 자동 탐색 후 로드
load_dotenv(find_dotenv())

# 환경변수 읽기
PUBLIC_AUTH_KEY = os.getenv("PUBLIC_AUTH_KEY")
WEATHER_AUTH_KEY = os.getenv("WEATHER_AUTH_KEY")

# print(WEATHER_AUTH_KEY)
# print(PUBLIC_AUTH_KEY)

## (1) 공공데이터포털에서 예보구역코드 조회
* 기상청_예보구역정보 조회서비스
* https://www.data.go.kr/data/15057111/openapi.do?recommendDataYn=Y

In [66]:
import requests, pandas as pd, io, os, json
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())                

url  = "http://apis.data.go.kr/1360000/FcstZoneInfoService/getFcstZoneCd"
params = {"serviceKey": PUBLIC_AUTH_KEY, "numOfRows": 3000, "pageNo": 1, "dataType": "JSON"}

resp = requests.get(url, params=params, timeout=10)
codes = resp.json()["response"]["body"]["items"]["item"]

zone_df = pd.json_normalize(codes)

In [69]:
pd.set_option('display.max_rows', None)

In [71]:
zone_df.tail(50)

,regId,lat,lon,regEn,regName,regSp,regUp,seq,stnF3,tmEd,tmSt
705,12C000MC,0.000000,0.000000,,울릉-독도,M,12C20000,1,0,210012310000,200507211500
706,12C10000,0.000000,0.000000,,동해남부,H,12C00000,1,0,210012310000,201701191130
707,12C10100,0.000000,0.000000,,동해남부앞바다,H,12C10000,1,0,210012310000,201701191130
708,12C10101,0.000000,0.000000,,울산앞바다,I,12C10100,1,0,210012310000,199001010000
709,12C10102,0.000000,0.000000,,경북남부앞바다,I,12C10100,1,0,210012310000,199001010000
710,12C10103,0.000000,0.000000,,경북북부앞바다,I,12C10100,1,0,210012310000,199001010000
711,12C10211,0.000000,0.000000,,동해남부남쪽안쪽먼바다,H,12C10000,1,0,210012310000,202107291100
712,12C10212,0.000000,0.000000,,동해남부북쪽안쪽먼바다,H,12C10000,1,0,210012310000,202107291100
713,12C10221,0.000000,0.000000,,동해남부남쪽바깥먼바다,H,12C10000,1,0,210012310000,202107291100
714,12C10222,0.000000,0.000000,,동해남부북쪽바깥먼바다,H,12C10000,1,0,210012310000,202107291100


In [67]:
zone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   regId    755 non-null    object 
 1   lat      755 non-null    float64
 2   lon      755 non-null    float64
 3   regEn    755 non-null    object 
 4   regName  755 non-null    object 
 5   regSp    755 non-null    object 
 6   regUp    755 non-null    object 
 7   seq      755 non-null    int64  
 8   stnF3    755 non-null    int64  
 9   tmEd     755 non-null    int64  
 10  tmSt     755 non-null    int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 65.0+ KB


## (2) 단기예보 육상 조회
* https://apihub.kma.go.kr/

In [53]:
import io
import requests
import pandas as pd
from datetime import datetime, timedelta

# tmfc 작성 (YYYYMMDDHHMM)
base_time = datetime.now().replace(hour=11, minute=0, second=0, microsecond=0)
tmfc = base_time.strftime("%Y%m%d%H%M")    # 예: 202506271100
print(tmfc)

# 파라미터 딕셔너리
params = {
    "authKey": WEATHER_AUTH_KEY,   # ✅ 필수
    "reg"   : "11B10101",   # 예보구역코드(서울·신촌) ― 없으면 전체
    "tmfc"  : tmfc,         # 발표시각 ― 없으면 가장 최근 자료
    "tmfc1" : tmfc,         # (선택) 기간 검색 시작
    "tmfc2" : tmfc,         # (선택) 기간 검색 끝
    "disp"  : 1,            # 0 = 변수별 컬럼, 1 = CSV 행 분리
}

url = "https://apihub.kma.go.kr/api/typ01/url/fct_afs_dl2.php"
resp = requests.get(url, params=params, timeout=10)
print(resp.status_code)
print(resp.headers.get("Content-Type"))

# 응답 본문의 문자 인코딩 형식을 EUC-KR로 설정 (디코딩 기준)
resp.encoding = "euc-kr" 
# print(resp.text)

# 줄 단위 분리
lines = resp.text.strip().splitlines()

# 헤더 찾기
header_line = next(line for line in lines if "REG_ID" in line)
columns = header_line.lstrip("#").strip().split()
print(columns)

# 데이터 줄 추출 (주석 제거 + 쉼표 개수로 판별)
data_lines = [
    line.rstrip(',=')
    for line in lines
    if not line.startswith("#") and line.count(",") >= 5
]
# print(data_lines)

# DataFrame 생성
df = pd.read_csv(
    io.StringIO("\n".join(data_lines)),
    header=None,
    names=columns
)
display(df.head())

202506281100
200
text/plain;charset=EUC-KR
['REG_ID', 'TM_FC', 'TM_EF', 'MOD', 'NE', 'STN', 'C', 'MAN_ID', 'MAN_FC', 'W1', 'T', 'W2', 'TA', 'STA', 'ST', 'SKY', 'PREP', 'WF']


,REG_ID,TM_FC,TM_EF,MOD,NE,STN,C,MAN_ID,MAN_FC,W1,T,W2,TA,STA,ST,SKY,PREP,WF
0,11B10101,202506281100,202506281200,A02,0,109,2,lan98,이혜란,S,1,SW,28,31,60,DB04,1,흐리고 가끔 비
1,11B10101,202506281100,202506290000,A02,1,109,2,lan98,이혜란,S,1,SW,23,26,60,DB04,1,흐리고 가끔 비
2,11B10101,202506281100,202506291200,A02,2,109,2,lan98,이혜란,S,1,SW,29,32,30,DB04,0,흐림
3,11B10101,202506281100,202506300000,A02,3,109,2,lan98,이혜란,S,1,SW,23,26,30,DB04,0,흐림
4,11B10101,202506281100,202506301200,A02,4,109,2,lan98,이혜란,SW,1,W,30,32,60,DB04,1,흐리고 한때 비


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   REG_ID  7 non-null      object
 1   TM_FC   7 non-null      int64 
 2   TM_EF   7 non-null      int64 
 3   MOD     7 non-null      object
 4   NE      7 non-null      int64 
 5   STN     7 non-null      int64 
 6   C       7 non-null      int64 
 7   MAN_ID  7 non-null      object
 8   MAN_FC  7 non-null      object
 9   W1      7 non-null      object
 10  T       7 non-null      int64 
 11  W2      7 non-null      object
 12  TA      7 non-null      int64 
 13  STA     7 non-null      int64 
 14  ST      7 non-null      int64 
 15  SKY     7 non-null      object
 16  PREP    7 non-null      int64 
 17  WF      7 non-null      object
dtypes: int64(10), object(8)
memory usage: 1.1+ KB
